# Using Swarm Ochestration

You can set the speaker selection method to `swarm` to use the swarm-like orchestration.

- **Headoffs**: Agents can transfer control to another agent via function calls, enabling smooth transitions within workflows.
- **Context Variables**: Agents can dynamically update shared variables through function calls, maintaining context and adaptability throughout the process.




## SwarmAgent and SwarmResult

To allow easier setup of the swarm, a `SwarmAgent` class is provided. When passing functions to this agent, the agent will execute any tool calls from LLMs directly. 

```python
agent = SwarmAgent(
    ..., # same arguments as AssistantAgent
    functions=[func_1, func_2, ...] # a new parameter that allows passing a list of functions
)
```

We also create a `SwarmResult` class, where you can return a value, the next agent to call and update context variables at the same time.

```python
def func_name() -> SwarmResult:
    return SwarmResult(value="value", agent=<swarmagent object>, context_variables={"key": "value"})
```

When defining function calls, you can return one of the following types:
- `str`
- `SwarmAgent`: if a `SwarmAgent` object is returned, it will be called next.
- `SwarmResult`: if a `SwarmResult` object is returned, we will use it to update the context variables, return a value and call the next agent.


**Notes for creating the function calls**
- For input arguments, you must define the type of the argument, otherwise the registration will fail (e.g. `arg_name: str`). 
- The docstring of the function will be used as the prompt. So make sure to write a clear description.
- The function name will be used as the tool name.


In [ ]:
from autogen.agentchat.swarm.swarm_agent import SwarmAgent, SwarmResult

llm_config = "Setup your config here"

def update_context_1(context_variables: dict) -> str:
    context_variables["1"] = True
    return SwarmResult(value="success", context_variables=context_variables)

def update_context_2_and_transfer_to_3(context_variables: dict) -> str:
    context_variables["2"] = True
    return SwarmResult(value="success", context_variables=context_variables, agent=agent_3)

def update_context_3(context_variables: dict) -> str:
    context_variables["3"] = True
    return SwarmResult(value="success", context_variables=context_variables)

def transfer_to_agent_2() -> SwarmAgent:
    return agent_2

agent_1 = SwarmAgent(
    name="Agent_1",
    system_message="You are Agent 1, first, call the function to update context 1, and transfer to Agent 2",
    llm_config=llm_config,
    functions=[update_context_1, transfer_to_agent_2],
)

agent_2 = SwarmAgent(
    name="Agent_2",
    system_message="You are Agent 2, call the function that updates context 2 and transfer to Agent 3",
    llm_config=llm_config,
    functions=[update_context_2_and_transfer_to_3],
)
agent_3 = SwarmAgent(
    name="Agent_3",
    system_message="You are Agent 3, first, call the function to update context 3, and then reply TERMINATE",
    llm_config=llm_config,
    functions=[update_context_3],
)

## Define a UserProxyAgent

The `Swarm` orchestration require human involvement. When a user (human) sends requests, the swarm would transfer controls within agents so that the most suitable agent can respond.

To use swarm, you need to define a `UserProxyAgent` class so that you can interact with the swarm. 

In [ ]:
from autogen import UserProxyAgent

user = UserProxyAgent(
    name="Human_User",
    system_message="Human user",
    human_input_mode="ALWAYS",
    code_execution_config=False,
)

## Start the GroupChat

Follow the GroupChat convention to define the groupchat and start the conversation.

You can pass in a `context_variables` dictionary to groupchat to initialize the context variables. Note this variable is only used for `swarm` orchestration.


In [ ]:
from autogen import GroupChat, GroupChatManager

context_variables = {"1": False, "2": False, "3": False}

groupchat = GroupChat(
    agents=[user, agent_1, agent_2, agent_3],
    messages=[],
    max_round=10,
    speaker_selection_method="swarm",
    context_variables=context_variables,
)
manager = GroupChatManager(groupchat=groupchat, llm_config=None)
chat_result = user.initiate_chat(manager, message="start")